Exe. 1

In [ ]:
#Import the MNIST dataset from keras.datasets and load it in x train, y train, x test, y test variables.
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#check the train and test shape.
print(X_train.shape)
print(X_test.shape)
#In oder to be able using the sigmoid activation function, normalize x train and x test according to the maximum and minimum elements of image set, for instance check x train[0].
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

Exe. 2

In [ ]:
#Plot some images to see your normalization results.
import matplotlib.pyplot as plt
plt.imshow(X_train[0], cmap='gray')
plt.show()
plt.imshow(X_train[1], cmap='gray')
plt.show()
plt.imshow(X_train[2], cmap='gray')
plt.show()
# Notice The Encoder generally uses a series of Dense and/or Convolutional layers to encode an 
# image into a fixed length vector that represents the image acompact form, while the Decoder 
# uses Dense and/or Convolutional layers toconvert the latent representation vector back into 
# that same image or anothermodified image (see Figure 1).

# Latent size is the size of the latent space: the vector holding the information
# after compression. This value is a crucial hyperparameter. If this value is too
# small, there won’t be enough data for reconstruction and if the value is too
# large, overfitting can occur.

Exe. 3

In [1]:
# Let’s define the LATENT SIZE = 32. Create an encoder model consists of a series  
# of dense layers, each layer is followed by a Dropout and a ReLU layer.
Latent_Size = 32
from keras.layers import Input, Dense, Dropout, Flatten, Reshape
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras import backend as K
input_img = Input(shape=(28, 28, 1))
x = Flatten()(input_img)
x = Dense(512)(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)
x = Dense(256)(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)
x = Dense(128)(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)
x = Dense(64)(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)
# Do not forget to add a final Dense layer for generating an output of LATENT SIZE.
encoded = Dense(Latent_Size)(x)
encoder = Model(input_img, encoded)
encoder.summary()

Exe. 4

Create a decoder model namely decoder. The decoder is essentially
the same as the encoder but in reverse.
Dense, ReLU, Dropout, Dense, ReLU, Dropout, Dense, ReLU,
Dropout, Dense, ReLU, Dropout, Dense, Acivation, Reshape
Notice that the Dense layers have the same size in the reverse order as the
encoder, and the final dense layer is a layer of size 28 × 28 = 784 that should
be reshaped to 28 × 28 for reproducing the same image input. The sigmoid
activation function output values in the range [0, 1] to fit with the input scaled
image data (use the Reshape from keras.layers).
To create the full model, the Keras Functional API must be used. The
Functional API allows us to string together multiple models.

In [ ]:
# Create a decoder model namely decoder. The decoder is essentially
# the same as the encoder but in reverse.
# Dense, ReLU, Dropout, Dense, ReLU, Dropout, Dense, ReLU,
# Dropout, Dense, ReLU, Dropout, Dense, Acivation, Reshape
decoder = Model(encoded, Flatten()(Dense(64)(LeakyReLU()(Dropout(0.3)(Dense(128)(LeakyReLU()(Dropout(0.3)(Dense(256)(LeakyReLU()(Dropout(0.3)(Dense(512)(LeakyReLU()(Dropout(0.3)(Dense(784)(LeakyReLU()(Dropout(0.3)(encoded)))))))))))))))))
decoder.summary()

# Notice that the Dense layers have the same size in the reverse order as the
# encoder, and the final dense layer is a layer of size 28 × 28 = 784 that should
# be reshaped to 28 × 28 for reproducing the same image input. The sigmoid
# activation function output values in the range [0, 1] to fit with the input scaled
# image data (use the Reshape from keras.layers).

# To create the full model, the Keras Functional API must be used. The
# Functional API allows us to string together multiple models.
from tensorflow.keras.layers import Input
img = Input(shape = (28,28))

# This will create a placeholder tensor which we can feed into each network to get
# the output of the whole model
latent_vector = encoder ( img )
output = decoder ( latent_vector )

# The best part about the Keras Functional API is how readable it is. The Keras
# Functional API allows you to call models directly onto tensors and get the
# output from that tensor. By calling the encoder model onto the img tensor, we
# get the latent vector. The same can be done  with the decoder model onto 
# the latent vector which gives us the output.
model = Model(inputs = img, outputs = output)
model.compile("nadam", loss = "binary_crossentropy")
model.summary()

# ’nadam’: Nesterov Adam optimizer. Much like Adam is essentially RMSprop
# with momentum, Nadam is RMSprop with Nesterov momentum. To create the
# model itself, we use the Model class and define what the inputs and outputs of
# the model are.